In [1]:
import numpy as np
import random
from pathlib import Path
# import progressbar as pb
import warnings # supress warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 20
pd.options.display.float_format = '{:.3f}'.format

In [2]:
df_items = pd.read_csv('Dataframes Brutos/DNC_order_items_dataset.csv').drop(columns='Unnamed: 0')
df_reviews = pd.read_csv('Dataframes Brutos/DNC_order_reviews_dataset.csv').drop(columns='Unnamed: 0')
df_orders = pd.read_csv('Dataframes Brutos/DNC_orders_dataset.csv').drop(columns='Unnamed: 0')
df_products = pd.read_csv('Dataframes Brutos/DNC_products_dataset.csv').drop(columns='Unnamed: 0')

In [3]:
df_items.head(-15)

,order_id,order_item_id,product_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.900
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.900
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.000
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.990
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.900
...,...,...,...,...
112630,fff6889749958e42b47a7977a4cf0ea0,1,75f6a4f019ec1322758d53b2fee2cc12,92.000
112631,fff6b8ca971f8e3ec822e99d0f2d3d21,1,9afaad66aca8b0c79e4f084a89c9c92b,199.000
112632,fff7c4452f050315db1b3f24d9df5fcd,1,dd469c03ad67e201bc2179ef077dcd48,736.000
112633,fff8286f77788ab8b55b2e5747fa7dd8,1,a2da86fa759178e9e58e54aa1a144e59,24.990


In [4]:
df_items.order_id.nunique(), df_reviews.order_id.nunique(), df_orders.order_id.nunique()

(98666, 98673, 99441)

In [5]:
df_reviews.head(15)

,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5
5,15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1
6,07f9bee5d1b850860defd761afa7ff16,e48aa0d2dcec3a2e87348811bcfdf22b,5
7,7c6400515c67679fbee952a7525281ef,c31a859e34e3adac22f376954e19b39d,5
8,a3f6f7f6f433de0aefbb97da197c554c,9c214ac970e84273583ab523dfafd09b,5
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4


In [6]:
df_orders.head(15)

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered
8,76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered
9,e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered


In [7]:
df_orders.order_id.nunique(), df_orders.customer_id.nunique()

(99441, 99441)

In [8]:
df_products.head(15)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.000,287.000,225.000,16.000,10.000,14.000
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.000,276.000,1000.000,30.000,18.000,20.000
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.000,250.000,154.000,18.000,9.000,15.000
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.000,261.000,371.000,26.000,4.000,26.000
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.000,402.000,625.000,20.000,17.000,13.000
5,41d3672d4792049fa1779bb35283ed13,instrumentos_musicais,60.000,745.000,200.000,38.000,5.000,11.000
6,732bd381ad09e530fe0a5f457d81becb,cool_stuff,56.000,1272.000,18350.000,70.000,24.000,44.000
7,2548af3e6e77a690cf3eb6368e9ab61e,moveis_decoracao,56.000,184.000,900.000,40.000,8.000,40.000
8,37cc742be07708b53a98702e77a21a02,eletrodomesticos,57.000,163.000,400.000,27.000,13.000,17.000
9,8c92109888e8cdf9d66dc7e463025574,brinquedos,36.000,1156.000,600.000,17.000,10.000,12.000


In [9]:
df_products.product_id.nunique()

32951

In [10]:
df_items.shape, df_reviews.shape, df_orders.shape, df_products.shape

((112650, 4), (99224, 3), (99441, 3), (32951, 8))

In [11]:
df_items.isnull().sum()

order_id         0
order_item_id    0
product_id       0
price            0
dtype: int64

In [54]:
df_items['order_item_id'].nunique()

21

In [13]:
df_reviews.isnull().sum()

review_id       0
order_id        0
review_score    0
dtype: int64

In [14]:
 df_orders.isnull().sum()

order_id        0
customer_id     0
order_status    0
dtype: int64

In [15]:
df_orders['order_status'].value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [16]:
df_products.isnull().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [55]:
df_products.dropna(inplace=True)

In [17]:
unic_customer = df_orders['customer_id'].nunique()
unic_customer

99441

In [18]:
unic_products = df_products['product_id'].nunique()
unic_products

32951

In [19]:
df = df_items.merge(df_orders, how='left', on='order_id').drop(columns='order_status')
df = df.merge(df_reviews, on='order_id', how='left').drop(columns='review_id')
df = df.merge(df_products, on = 'product_id', how='left')

In [20]:
df.isnull().sum()

order_id                         0
order_item_id                    0
product_id                       0
price                            0
customer_id                      0
review_score                   942
product_category_name         1612
product_name_lenght           1612
product_description_lenght    1612
product_weight_g                18
product_length_cm               18
product_height_cm               18
product_width_cm                18
dtype: int64

In [22]:
df.shape

(110773, 13)

In [23]:
df.head(20)

,order_id,order_item_id,product_id,price,customer_id,review_score,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.900,3ce436f183e68e07877b285a838db11a,5.000,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.900,f6dd3ec061db4e3987629fe6b26e5cce,4.000,pet_shop,56.000,239.000,30000.000,50.000,30.000,40.000
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.000,6489ae5e4333f3693df5ad4372dab6d3,5.000,moveis_decoracao,59.000,695.000,3050.000,33.000,13.000,33.000
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.990,d4eb9395c8c0431ee92fce09860c5a06,4.000,perfumaria,42.000,480.000,200.000,16.000,10.000,15.000
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.900,58dbd0b2d70206bf40e62cd34e84d795,5.000,ferramentas_jardim,59.000,409.000,3750.000,35.000,40.000,30.000
5,00048cc3ae777c65dbb7d2a0634bc1ea,1,ef92defde845ab8450f9d70c526ef70f,21.900,816cbea969fe5b689b39cfc97a506742,4.000,utilidades_domesticas,36.000,558.000,450.000,24.000,8.000,15.000
6,00054e8431b9d7675808bcb819fb4a32,1,8d4f2bb7e93e6710a28f34fa83ee7d28,19.900,32e2e6ab09e778d99bf2e0ecd4898718,4.000,telefonia,52.000,815.000,200.000,27.000,5.000,20.000
7,000576fe39319847cbb9d288c5617fa6,1,557d850972a7d6f792fd18ae1400d9b6,810.000,9ed5e522dd9dd85b4af4a077526d8117,5.000,ferramentas_jardim,39.000,1310.000,13805.000,35.000,75.000,45.000
8,0005a1a1728c9d785b8e2b08b904576c,1,310ae3c140ff94b03219ad0adc3c778f,145.950,16150771dfd4776261284213b89c304e,1.000,beleza_saude,59.000,493.000,2000.000,30.000,12.000,16.000
9,0005f50442cb953dcd1d21e1fb923495,1,4535b0e1091c278dfd193e5a1d63b39f,53.990,351d3cb2cee3c7fd0af6616c82df21d3,4.000,livros_tecnicos,52.000,1192.000,850.000,29.000,3.000,21.000


In [24]:
df.sort_values(by='order_item_id', ascending=False)

,order_id,order_item_id,product_id,price,customer_id,review_score,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
57686,8272b63d03f5f79c56e9e4120aec44ef,21,79ce45dbc2ea29b22b5a261bbb7b7ee7,7.800,fc3d1daec319d62d49bfb5e1f83123e9,1.000,beleza_saude,27.000,152.000,1000.000,25.000,6.000,12.000
12028,1b15974a0141d54e36626dca3fdc731a,20,ee3d532c8a438679776d222e997606b3,100.000,be1b70680b9f9694d8c70f41fa3dc92b,1.000,informatica_acessorios,43.000,452.000,360.000,19.000,18.000,15.000
57685,8272b63d03f5f79c56e9e4120aec44ef,20,270516a3f41dc035aa87d220228f844c,1.200,fc3d1daec319d62d49bfb5e1f83123e9,1.000,beleza_saude,45.000,232.000,800.000,21.000,4.000,15.000
57684,8272b63d03f5f79c56e9e4120aec44ef,19,270516a3f41dc035aa87d220228f844c,1.200,fc3d1daec319d62d49bfb5e1f83123e9,1.000,beleza_saude,45.000,232.000,800.000,21.000,4.000,15.000
12027,1b15974a0141d54e36626dca3fdc731a,19,ee3d532c8a438679776d222e997606b3,100.000,be1b70680b9f9694d8c70f41fa3dc92b,1.000,informatica_acessorios,43.000,452.000,360.000,19.000,18.000,15.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40284,5b1ac5be77fb87f6f96cc73e47887575,1,94976cc078d10e74c19c62f0b3b52432,139.000,e3d5f8908d3b4fb38cb028a3b8849966,5.000,cama_mesa_banho,48.000,262.000,6150.000,52.000,16.000,44.000
40283,5b19e6d8d588e5f729477c3df4552051,1,440f5b159d6123c0f65eb1abb5373b99,159.900,5252326c6b990fdcde424e82efe2a7ca,5.000,papelaria,58.000,264.000,2200.000,30.000,60.000,19.000
40282,5b19cc30f1a0a44439a744db5c2cc979,1,bb48c8fda6bcf73c9a0178fd1a1a366d,44.900,1204a88c6f98fdc14edcbc6dc6df434c,5.000,cama_mesa_banho,59.000,58.000,400.000,20.000,2.000,15.000
40281,5b19559d742224610c730e73362fd0e4,1,dd6a505f83dd3c6326aa9856519e0978,49.900,48978b3b05f04639d393318bb74c459b,5.000,brinquedos,43.000,623.000,200.000,16.000,7.000,11.000


In [57]:
df.product_category_name.value_counts()

cama_mesa_banho                  11137
beleza_saude                      9645
esporte_lazer                     8640
moveis_decoracao                  8331
informatica_acessorios            7849
                                 ...  
cds_dvds_musicais                   14
la_cuisine                          13
pc_gamer                             9
fashion_roupa_infanto_juvenil        8
seguros_e_servicos                   2
Name: product_category_name, Length: 73, dtype: int64

In [25]:
df.to_csv('df_completo.csv', index=False)

In [26]:
%%writefile data.py
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


def get_data(path):
  return pd.read_csv(path)


def start_pipeline(dataf):
  return dataf.copy()


def get_product_count(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'order_id': pd.Series.nunique}) \
    .rename(columns={'order_id': 'order_id_count'}) \
    .set_index('product_id') \
    .sort_values('order_id_count', ascending = False)


def get_n_top_values(dataf, n):
  return dataf.head(n) \
    .index \
    .tolist()


def format_sample(dataf):
  return pd.DataFrame({
      "pedidos": dataf["order_id"],
      "produto": dataf["product_id"],
      "action": dataf["review_score"]
  })


def get_top_products_ids(dataf, sample_size=1000):
  return dataf \
    .pipe(start_pipeline) \
    .pipe(get_product_count) \
    .pipe(get_n_top_values, sample_size)


def get_product_based_similarity_matrix(data):
  top_product_ids = get_top_products_ids(data)
  product_sampled_relationship_data = data[data["product_id"].isin(top_product_ids)] \
    .pipe(format_sample) \
    .pivot_table(index="pedidos", columns="produto", values="action").fillna(0)
  return cosine_similarity(product_sampled_relationship_data.T), product_sampled_relationship_data.T.index

Overwriting data.py


In [27]:
%%writefile product_based_recommender.py
import random

import numpy as np

from data import get_data, get_product_based_similarity_matrix


class ProductBasedRecommender:

  def __init__(self, data_path):
    self.data_path = data_path

  def setup(self):
    data = get_data(self.data_path)
    sim_matrix, available_products_index = get_product_based_similarity_matrix(data)
    self.available_products_index = available_products_index
    self.sim_matrix = sim_matrix

  def get_random_product_index(self):
    product_id = self.available_products_index[random.randint(0, self.sim_matrix.shape[0])]
    return product_id

  def get_n_closest(self, product_id, n=10):
    if product_id not in self.available_products_index:
      raise ValueError('Product is not available on data.')
    product_index = self.available_products_index.tolist().index(product_id)
    closest_n_products = np.argsort(-self.sim_matrix[product_index])[1:n]
    n_closest = []
    for product_data in zip(self.available_products_index[closest_n_products], self.sim_matrix[product_index][closest_n_products]):
      n_closest.append(product_data)
    return tuple(n_closest)

Overwriting product_based_recommender.py


In [28]:
df2 = pd.read_csv('df_completo.csv')

In [29]:
df2.head()

,order_id,order_item_id,product_id,price,customer_id,review_score,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.900,3ce436f183e68e07877b285a838db11a,5.000,cool_stuff,58.000,598.000,650.000,28.000,9.000,14.000
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.900,f6dd3ec061db4e3987629fe6b26e5cce,4.000,pet_shop,56.000,239.000,30000.000,50.000,30.000,40.000
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.000,6489ae5e4333f3693df5ad4372dab6d3,5.000,moveis_decoracao,59.000,695.000,3050.000,33.000,13.000,33.000
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.990,d4eb9395c8c0431ee92fce09860c5a06,4.000,perfumaria,42.000,480.000,200.000,16.000,10.000,15.000
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.900,58dbd0b2d70206bf40e62cd34e84d795,5.000,ferramentas_jardim,59.000,409.000,3750.000,35.000,40.000,30.000


In [30]:
from pathlib import Path

from product_based_recommender import ProductBasedRecommender

DATA_PATH = Path(".")

complete_df_path = DATA_PATH / 'df_completo.csv'

product_recommender = ProductBasedRecommender(complete_df_path)

product_recommender.setup()

# Utilizando a recomendação

In [53]:
some_product_id = product_recommender.get_random_product_index()



product_recommender.get_n_closest(some_product_id)

(('f4d705aa95ccca448e5b0deb6e5290ba', 0.13553401729778453),
 ('4025ee582ef6b8c478af3b44cf89054b', 0.07595648672650816),
 ('31c1d1dfbb4b2730140b8bd9248bcd58', 0.05967849236638964),
 ('8a443635fdf9759915c9be5be2e3b862', 0.04231992340618113),
 ('e0cf79767c5b016251fe139915c59a26', 0.011800893468634785),
 ('aef8207c2e91f4fcdbc0824737abf358', 0.0),
 ('a92930c327948861c015c919a0bcb4a8', 0.0),
 ('a9516a079e37a9c9c36b9b78b10169e8', 0.0),
 ('a9f480bfc49f0b85e8394e1cdf685b4e', 0.0))